# Pre-processing pipeline

Notebook used to compute the pre-processing pipeline on the raw data coming from one deployment.

In [3]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import importlib
import scripts.theodolite_utils as theodo_u
import scripts.theodolite_function as theodo_f
import scripts.prediction_utils as prediction_u
import time
from tqdm import tqdm
from stheno import B
import torch
# import matplotlib.pyplot as plt
# from scipy.interpolate import splprep, splev
# from scipy import interpolate
# from scipy import spatial
# import seaborn as sns
# from scipy.spatial.transform import Rotation as R
# from IPython.display import HTML
# from sklearn.gaussian_process import GaussianProcessRegressor
# from sklearn.gaussian_process.kernels import (RBF, Matern, RationalQuadratic, ExpSineSquared, DotProduct, ConstantKernel)
# from mpl_toolkits.mplot3d.art3d import Poly3DCollection
# from wbml.plot import tweak

ModuleNotFoundError: No module named 'stheno'

In [27]:
# Selection of files and parameters

# Path of the rosbags
file = [
        "/home/maxime/data/IROS_2022/20220224_TS/2022-02-24-15-34-38.bag"
       ]

# Path of output
output = [
        "./data/20220224/"
]

# Parameters to select:
# 1. Apply filtering or not (Module 1)
# 2-3-4. Parameters tau_r, tau_a, tau_e (Module 1)
# 5. Parameter tau_s (Module 2)
# 6. Parameter tau_l (Module 3)
# 7. Parameter to cut edge of sub-intervals (former parameter with low impact)
parameters = [
    [1,2,1,1,1,6,0],
]

In [34]:
theodo_u = importlib.reload(theodo_u)
theodo_f = importlib.reload(theodo_f)
prediction_u = importlib.reload(prediction_u)

for param in parameters:
    print(param)

    if(param[0]==0):
        filtering = False
    if(param[0]==1):
        filtering = True
    thresold_d = param[1]                # tau_r [m/s]
    thresold_a = param[2]                # tau_a [deg/s]
    thresold_e = param[3]                # tau_e [deg/s]
    limit_time_interval = param[4]       # tau_s [s]

    B.epsilon = 1e-8
    Mode = "SGP"       # Interpolation choice: 1. L -> Linear interpolation, 2. SGP -> Gaussian Process with Stheno library
    limit_search = limit_time_interval
    size_interval = param[5]         # tau_l [s]
    delta_t = param[6]               # Value to remove points near edge of time intervals [s]
    save = True

    # GP parameters
    verbose=False
    Number_restart = 100        # Number of restart for the GP
    noise_GP = 0                # Noise of GP
    variance_GP = 1             # Variance of GP
    lengthscale_GP = 1          # Lengthscale of GP

    save_index_1 = []

    for fname, opath in zip(file,output): 
        if(not filtering):
            path = opath + "raw_prediction/"
        else:
            path = opath + "filtered_prediction/"

        if(filtering):
            t1, t2, t3, tp1, tp2, tp3, d1, d2, d3, a1, a2, a3, e1, e2, e3 = theodo_u.read_rosbag_theodolite_without_tf_raw_data_pre_filtered(fname)
            index_1_f = theodo_u.thresold_raw_data(t1, d1, a1, e1, thresold_d, thresold_a*3.1415926/180, thresold_e*3.1415926/180, limit_time_interval)
            index_2_f = theodo_u.thresold_raw_data(t2, d2, a2, e2, thresold_d, thresold_a*3.1415926/180, thresold_e*3.1415926/180, limit_time_interval)
            index_3_f = theodo_u.thresold_raw_data(t3, d3, a3, e3, thresold_d, thresold_a*3.1415926/180, thresold_e*3.1415926/180, limit_time_interval)
            t1 = t1[index_1_f]
            t2 = t2[index_2_f]
            t3 = t3[index_3_f]
            tp1 = tp1[index_1_f].T
            tp2 = tp2[index_2_f].T
            tp3 = tp3[index_3_f].T
            print(len(t1),len(t2),len(t3))
        else:
            t1, t2, t3, tp1, tp2, tp3, d1, d2, d3, a1, a2, a3, e1, e2, e3 = theodo_u.read_rosbag_theodolite_without_tf_raw_data(fname)
            print(len(t1),len(t2),len(t3))

        time_origin = np.min([t1[0],t2[0],t3[0]])

        start_time = time.time()

        t1 = t1 - np.ones_like(t1)*time_origin
        t2 = t2 - np.ones_like(t2)*time_origin
        t3 = t3 - np.ones_like(t3)*time_origin

        list_interval, list_time = theodo_f.split_time_interval_all_data(t1, t2, t3, limit_time_interval)
        list_trajectories_split = theodo_f.merge_interval(list_interval, list_time, t1, t2, t3, limit_search)

        Prediction_1 = []
        Prediction_2 = []
        Prediction_3 = []
        T_prediction = []

        for i in tqdm(list_trajectories_split):

            index_1 = np.array([i[0,0],i[1,0]])
            index_2 = np.array([i[0,1],i[1,1]])
            index_3 = np.array([i[0,2],i[1,2]])

            save_index_1.append(index_1)

            if(filtering==True):
                index_1 = prediction_u.delta_t_function(index_1,t1,delta_t)
                index_2 = prediction_u.delta_t_function(index_2,t2,delta_t)
                index_3 = prediction_u.delta_t_function(index_3,t3,delta_t)

            begin = np.max([t1[index_1[0]], t2[index_2[0]], t3[index_3[0]]])
            end = np.min([t1[index_1[1]], t2[index_2[1]], t3[index_3[1]]])

            if(abs(end-begin)>size_interval and begin<end):

                rate = 10  #Hz
                T_prediction_init = torch.from_numpy(np.arange(begin, end, 1/rate))

                if(Mode == "MGPO" or Mode == "All"):
                    T_MGPO, S_MGPO = prediction_u.data_training_MGPO(t1, t2, t3, tp1, tp2, tp3, index_1, index_2, index_3)
                    print("Training")
                    m = prediction_u.training_MGPO(Number_restart, verbose, T_MGPO, S_MGPO)
                    print("Prediction")
                    for i in T_prediction_init.numpy():
                        T_prediction.append(i+time_origin)
                        P1_MGPO, P2_MGPO, P3_MGPO = prediction_u.unit_prediction_MGPO(i, m)
                        Prediction_1.append(P1_MGPO)
                        Prediction_2.append(P2_MGPO)
                        Prediction_3.append(P3_MGPO)

                if(Mode == "GP" or Mode == "All"):
                    T1, X1, Y1, Z1, T2, X2, Y2, Z2, T3, X3, Y3, Z3 = prediction_u.data_training_GP(t1, t2, t3, tp1, tp2, tp3, index_1, index_2, index_3)
                    print("Training")
                    mx1, my1, mz1, mx2, my2, mz2, mx3, my3, mz3 = prediction_u.training_GP(Number_restart, verbose, T1, X1, Y1, Z1, T2, X2, Y2, Z2, T3, X3, Y3, Z3)
                    print("Prediction")
                    for i in T_prediction_init.numpy():
                        T_prediction.append(i+time_origin)
                        P1_GP, P2_GP, P3_GP = prediction_u.unit_prediction_GP(i, mx1, my1, mz1, mx2, my2, mz2, mx3, my3, mz3)
                        Prediction_1.append(P1_GP)
                        Prediction_2.append(P2_GP)
                        Prediction_3.append(P3_GP)

                if(Mode == "L" or Mode == "All"):
                    T1, X1, Y1, Z1, T2, X2, Y2, Z2, T3, X3, Y3, Z3 = prediction_u.data_training_L(t1, t2, t3, tp1, tp2, tp3, index_1, index_2, index_3)
                    print("Prediction")
                    mx1, my1, mz1, mx2, my2, mz2, mx3, my3, mz3 = prediction_u.linear_interpolation(T1, X1, Y1, Z1, T2, X2, Y2, Z2,T3, X3, Y3, Z3)

                    for i in T_prediction_init.numpy():
                        T_prediction.append(i+time_origin)
                        P1_GP, P2_GP, P3_GP = prediction_u.linear_prediction(i, time_origin, mx1, my1, mz1, mx2, my2, mz2, mx3, my3, mz3)
                        Prediction_1.append(P1_GP)
                        Prediction_2.append(P2_GP)
                        Prediction_3.append(P3_GP)

                if(Mode=="SGP"):
                    prediction_value = T_prediction_init.numpy()
                    # Prepare data for training
                    T1, X1, Y1, Z1, T2, X2, Y2, Z2, T3, X3, Y3, Z3 = prediction_u.data_training_GP_stheno(t1, t2, t3, tp1, tp2, tp3, index_1, index_2, index_3)
                    # Training for each axis
                    m_X1, v_X1 = prediction_u.GP_function_stheno(x=prediction_value, x_obs=T1, y_obs=X1, variance=variance_GP, lengthscale=lengthscale_GP, noise_init=noise_GP, optimization_nb=Number_restart)
                    m_Y1, v_Y1 = prediction_u.GP_function_stheno(x=prediction_value, x_obs=T1, y_obs=Y1, variance=variance_GP, lengthscale=lengthscale_GP, noise_init=noise_GP, optimization_nb=Number_restart)
                    m_Z1, v_Z1 = prediction_u.GP_function_stheno(x=prediction_value, x_obs=T1, y_obs=Z1, variance=variance_GP, lengthscale=lengthscale_GP, noise_init=noise_GP, optimization_nb=Number_restart)
                    m_X2, v_X2 = prediction_u.GP_function_stheno(x=prediction_value, x_obs=T2, y_obs=X2, variance=variance_GP, lengthscale=lengthscale_GP, noise_init=noise_GP, optimization_nb=Number_restart)
                    m_Y2, v_Y2 = prediction_u.GP_function_stheno(x=prediction_value, x_obs=T2, y_obs=Y2, variance=variance_GP, lengthscale=lengthscale_GP, noise_init=noise_GP, optimization_nb=Number_restart)
                    m_Z2, v_Z2 = prediction_u.GP_function_stheno(x=prediction_value, x_obs=T2, y_obs=Z2, variance=variance_GP, lengthscale=lengthscale_GP, noise_init=noise_GP, optimization_nb=Number_restart)
                    m_X3, v_X3 = prediction_u.GP_function_stheno(x=prediction_value, x_obs=T3, y_obs=X3, variance=variance_GP, lengthscale=lengthscale_GP, noise_init=noise_GP, optimization_nb=Number_restart)
                    m_Y3, v_Y3 = prediction_u.GP_function_stheno(x=prediction_value, x_obs=T3, y_obs=Y3, variance=variance_GP, lengthscale=lengthscale_GP, noise_init=noise_GP, optimization_nb=Number_restart)
                    m_Z3, v_Z3 = prediction_u.GP_function_stheno(x=prediction_value, x_obs=T3, y_obs=Z3, variance=variance_GP, lengthscale=lengthscale_GP, noise_init=noise_GP, optimization_nb=Number_restart)

                    # Save prediction at time i
                    for i,mx,my,mz,vx,vy,vz in zip(T_prediction_init.numpy(), m_X1, m_Y1, m_Z1, v_X1, v_Y1, v_Z1):
                        Prediction_1.append(np.array([i+time_origin, mx ,my, mz, vx, vy, vz]))
                        T_prediction.append(i+time_origin)
                    for i,mx,my,mz,vx,vy,vz in zip(T_prediction_init.numpy(), m_X2, m_Y2, m_Z2, v_X2, v_Y2, v_Z2):
                        Prediction_2.append(np.array([i+time_origin, mx ,my, mz, vx, vy, vz]))
                    for i,mx,my,mz,vx,vy,vz in zip(T_prediction_init.numpy(), m_X3, m_Y3, m_Z3, v_X3, v_Y3, v_Z3):
                        Prediction_3.append(np.array([i+time_origin, mx ,my, mz, vx, vy, vz]))

        stop_time = time.time()
        print(stop_time - start_time)

        print("Interpolation finished !")

        if save:
            if(Mode == "MGPO" or Mode == "All"):
                if(filtering):
                    trajectoire = "f-"+str(thresold_d)+"-"+str(thresold_a)+"-"+str(thresold_e)+"-"+str(limit_time_interval)+"-"+str(size_interval)+"-"+str(delta_t)+"-MGPO-"+str(Number_restart)
                else:
                    trajectoire = "nf-"+str(limit_time_interval)+"-"+str(size_interval)+"-"+str(delta_t)+"-MGPO-"+str(Number_restart)

                theodo_u.Convert_data_prediction_to_csv(T_prediction, Prediction_1, path+trajectoire+ "_1.csv")
                theodo_u.Convert_data_prediction_to_csv(T_prediction, Prediction_2, path+trajectoire+ "_2.csv")
                theodo_u.Convert_data_prediction_to_csv(T_prediction, Prediction_3, path+trajectoire+ "_3.csv")
            if(Mode == "GP" or Mode == "All"):
                if(filtering):
                    trajectoire = "f-"+str(thresold_d)+"-"+str(thresold_a)+"-"+str(thresold_e)+"-"+str(limit_time_interval)+"-"+str(size_interval)+"-"+str(delta_t)+"-GP-"+str(Number_restart)
                else:
                    trajectoire = "nf-"+str(limit_time_interval)+"-"+str(size_interval)+"-"+str(delta_t)+"-GP-"+str(Number_restart)

                theodo_u.Convert_data_prediction_to_csv(T_prediction, Prediction_1, path+trajectoire+ "_1.csv")
                theodo_u.Convert_data_prediction_to_csv(T_prediction, Prediction_2, path+trajectoire+ "_2.csv")
                theodo_u.Convert_data_prediction_to_csv(T_prediction, Prediction_3, path+trajectoire+ "_3.csv")
            if(Mode == "SGP" or Mode == "All"):
                if(filtering):
                    trajectoire = "f-"+str(thresold_d)+"-"+str(thresold_a)+"-"+str(thresold_e)+"-"+str(limit_time_interval)+"-"+str(size_interval)+"-"+str(delta_t)+"-SGP-"+str(Number_restart)
                else:
                    trajectoire = "nf-"+str(limit_time_interval)+"-"+str(size_interval)+"-"+str(delta_t)+"-SGP-"+str(Number_restart)

                theodo_u.Convert_data_prediction_to_csv(T_prediction, Prediction_1, path+trajectoire+ "_1.csv")
                theodo_u.Convert_data_prediction_to_csv(T_prediction, Prediction_2, path+trajectoire+ "_2.csv")
                theodo_u.Convert_data_prediction_to_csv(T_prediction, Prediction_3, path+trajectoire+ "_3.csv")
            if(Mode == "L" or Mode == "All"):
                if(filtering):
                    trajectoire = "f-"+str(thresold_d)+"-"+str(thresold_a)+"-"+str(thresold_e)+"-"+str(limit_time_interval)+"-"+str(size_interval)+"-"+str(delta_t)+"-L"
                else:
                    trajectoire = "nf-"+str(limit_time_interval)+"-"+str(size_interval)+"-"+str(delta_t)+"-L"

                if save:
                    theodo_u.Convert_raw_data_predicted_to_csv(T_prediction, Prediction_1, path+trajectoire+ "_1.csv")
                    theodo_u.Convert_raw_data_predicted_to_csv(T_prediction, Prediction_2, path+trajectoire+ "_2.csv")
                    theodo_u.Convert_raw_data_predicted_to_csv(T_prediction, Prediction_3, path+trajectoire+ "_3.csv")

        print("Saved !")

[1, 2, 1, 1, 1, 0, 6]
Number of data for theodolites: [4945 4681 4737]
Bad measures: 558
3514 3329 3414


  0%|          | 0/92 [00:00<?, ?it/s]


NotFoundLookupError: For function "to_tensor", signature Signature(torch.Tensor) could not be resolved.

In [ ]:
ttf = importlib.reload(ttf)
tfu = importlib.reload(tfu)

k = './data/prediction/TS/20220717/02/'
path_type = 'raw/'
path_file_type = 'nf-1-6-1-L_'

trimble_1 = ttf.read_prediction_data_resection_csv_file(k+path_type+path_file_type+"1.csv")
t1, t2, t3, tp1, tp2, tp3, d1, d2, d3, a1, a2, a3, e1, e2, e3 = ttf.read_rosbag_theodolite_without_tf_raw_data("/home/norlab/Data/IROS_2022/20220717_TS/constrained_2022-07-17-20-11-10.bag")

In [ ]:
%matplotlib notebook

save_index_1_arr = np.array(save_index_1)
i = save_index_1_arr[20]

plt.figure()

plt.scatter(t1,tp1[0,:],s =1)
#plt.scatter(t2,tp2[0,:],s =1)

plt.scatter(t1[i[0]:i[1]],tp1[0,i[0]:i[1]], linewidth=5, color= 'black')

#plt.plot(trimble_1[i[0,0]:i[1,0],0],trimble_1[i[0,0]:i[1,0],1], linewidth=5, color= 'black')
#plt.plot(trimble_1[i[0,0]:i[1,0],0],trimble_1[i[0,0]:i[1,0],1], linewidth=5, color= 'black')
plt.scatter(trimble_1[:,0],trimble_1[:,1], linewidth=2, color= 'red')


plt.show()